## Luke Waninger
#### 25 May 2018 

In [3]:
from H7_source import *
from IPython.core.interactiveshell import InteractiveShell
from sklearn.datasets import load_digits
from sklearn.decomposition import PCA
from sklearn.metrics import pairwise
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

InteractiveShell.ast_node_interactivity = 'all'

exp   = np.exp
ident = np.identity
na    = np.newaxis
norm  = np.linalg.norm
np.random.seed(42)

### The gradient $\triangledown F(\alpha)$ of $F$
for a kernel based support vector machine can be shown to be...

$\triangledown F(\alpha) = 1/n \sum_{i=1}^n \triangledown l(y_i, (K\alpha)_i) + 2\lambda K \alpha$



$
\triangledown l(y,t) =
	\begin{cases}
        0 \hspace{55pt}  yt > 1 + h \\
        -y_i x_i \frac{1+h-yt}{2h} \hspace{18pt}  |1-yt| \le h \\
        -y_i x_i \hspace{40pt}  yt < 1-h
	\end{cases}
$

### Computing the graham and kernel matrices for a Gaussian (RBF) and polynomial kernel
I decided to encapsulate these functions into one class each for radial and polynomial kernels for readability and code reuse. The $\texttt{compute}$ function calculates and returns the requested kernel for any set of observations.

In [4]:
class k_radialrbf(Kernel):
    def __init__(self, sigma):
        super().__init__()
        self.sigma = sigma

    def __str__(self):
        return f'rbf({self.sigma})'

    def compute(self, x, xp=None):
        sigma = self.sigma
        xp = x if xp is None else xp

        def norm(mat):
            return np.linalg.norm(mat, axis=1)

        return exp(-1/(2*sigma**2) * ((norm(x)** 2)[:, na] + (norm(xp)**2)[na, :]-2*(x @ xp.T)))


class k_polynomial(Kernel):
    def __init__(self, degree, b=1.):
        super().__init__()
        self.degree = degree
        self.b = b

    def __str__(self):
        return f'polynomial({self.degree})'

    def compute(self, x, xp=None):
        xp = x if xp is None else xp

        return (x @ xp.T + self.b)**self.degree

### Using scikit learn's builtin Digits dataset, download and standardize.

In [5]:
x, y = load_digits(n_class=10, return_X_y=True)

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.33, random_state=42)

scalar  = StandardScaler().fit(x_train)
x_train = scalar.transform(x_train)
x_test  = scalar.transform(x_test)

### The support vector machine: $\texttt{mysvm}$
Again, I chose to implement this using OOP so I could encapsulate the SVM, and other helper functions in a single object.

In [6]:
class MySVM(Estimator):
    def __init__(self, kernel):
        self.kernel = kernel
    
    def __str__(self):
        return f'SVM(kernel={self.kernel})'
    
    @staticmethod
    def gradient(k, y, beta, l, h=0.5):
        n, d = k.shape
        lg = np.zeros([n, d])

        yk = y *(k @ beta)
        mask = np.abs(1 - yk)

        lg[mask <= h] = ((1/(2*h)) * ((1 + h-yk)[:, na]) * (-y[:, na] * k))[mask <= h]
        lg[yk < 1-h]  = (-y[:, na] * k)[yk < 1-h]

        return np.array(np.sum(lg, axis=0)/n + 2*l*beta)
    
    def fgrad(self, k, y, l, eta=1., max_iter=100):
        n, d  = k.shape
        b0    = np.zeros(d)
        theta = np.copy(b0)
        grad  = self.gradient(k, y, b0, l)

        i = 0
        while i < max_iter and not np.isclose(0, eta):
            eta = backtracking(k, y, b0, l, eta, self.gradient, self.objective)

            b1 = theta - eta*grad
            theta = b1 + (i/(i+3))*(b1-b0)
            grad  = self.gradient(k, y, theta, l)
            b0 = b1

            i += 1

        return b0
    
    @staticmethod
    def objective(k, y, l, beta, h=0.5):
        n, d = k.shape
        loss = np.zeros(n)
        yk = y * (k @ beta)
        mask = np.abs(1 - yk)

        loss[mask <= h] = ((1 + h-yk)**2 / (4*h))[mask <= h]
        loss[yk < 1-h] = (1 - yk)[yk < 1-h]

        return np.sum(loss)/n + l*norm(beta)**2

    def predict(self, kp, beta):
        return [1 if ki @ beta.T > 0 else -1 for ki in kp]
    
    def predict_proba(self, kp, beta):
        return [ki @ beta.T for ki in kp]

### Training the SVM with the huberized hinge loss and an order 7 polynomial kernel

running one vs rest with a polynomial kernel of degree 7 with $\lambda$=1 gives a horrible validation error: $\approx$ 0.518

In [7]:
kernl = k_polynomial(7)
OVR(MySVM(kernl), n_jobs=-1).fit(x_train, y_train, x_test, y_test, 1.)

AttributeError: Can't pickle local object 'track_bar.<locals>.track_it'

using cross-validation we see close to a performance improvement of around 9%

In [8]:
ovr = OVR(MySVM(kernl), n_jobs=-1)
cv(x_train, y_train, ovr, eargs=np.linspace(.001, 1., 5), nfolds=3)

### compare the performance of kernel SVMs
It quickly becomes clear the 7-degree polynomial kernel is a horrible choice. Below, I run a series of OVO polynomial and radial kernels that all show much better performance. 

In [7]:
kernels = [
    k_polynomial(1),
    k_polynomial(3),
    k_polynomial(5),
    k_radialrbf(1),
    k_radialrbf(5),
    k_radialrbf(10)
]

for kernl in kernels:
    ovr, eargs = OVR(MySVM(kernl), n_jobs=-1), np.linspace(.001, 1., 5)
    cv(x_train, y_train, ovr, eargs, nfolds=3)

## checking my OvO

## Exercise 2

In [ ]:
def oja(x, t=1., max_iter=50):    
    n, d = x.shape
    w1  = np.random.normal(size=d)
    w1 /= norm(w1)

    for i in range(max_iter):
        w1 = w1 + t*(x.T @ x @ w1)
        w1 = w1/norm(w1)
        t  = 1/(i+1)

    w1 = w1[:, na]
    C  = x.T @ x @ (ident(d) - w1 @ w1.T)
    
    w2 = np.random.normal(size=d)
    w2 = w2/norm(w2)
    for i in range(max_iter):
        w2 = w2 + t * (C @ w2)
        w2 = w2/norm(w2)
        t  = 1/(i+1)

    w2 = w2[:, na]
    return np.concatenate([x@w1, x@w2], axis=1)

In [ ]:
def oja_r(wp, n, t, max_iter):
    if n == end:
        return None
    
    n, d = x.shape
    w = np.random.norm(size=d)
    w /= norm(w1)
    
    C  = x.T @ x @ (ident(d) - wp @ wp.T)
    for i in range(max_iter):
        w = w + t*(x.T @ x @ w)
        w = w/norm(w)
        t = 1/(i+3)
    
    w = w[:, na]
    return np.concatentate([x@w, oja_r(w, n+1, t, max_iter)], axis=1).flatten()

### generate a simulated dataset

In [ ]:
def gen_k(n, m, scale):
    def gen_ki(m, s):
        return np.random.normal(m, s, 1)
    
    start = n*scale
    means = np.random.uniform(start, start+m, m)
    np.random.shuffle(means)

    return np.array([[gen_ki(mi, 25) for mi in means] for i in range(n)]).reshape((n, m))

### run my oja and plot vs scikit's

In [ ]:
n_obs, n_feat, n_klass = 30, 60, 3
x = np.array([gen_k(n_obs, n_feat, i) for i in range(n_klass)]).reshape((n_obs*n_klass, n_feat))

mykit = oja(x)
scikt = PCA().fit_transform(X=x)

pca_plot([mykit, scikt], n_obs, n_klass)